# Can we classify open clusters and globular clusters using their B-V index and true diameter?

## Abstract

## Introduction

## Reading the Data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm


In [2]:
def get_angular_size(r_h):
	'''Go  from half light radius to true radius, based on King (1966) model for half light radius.
	Takes half light radius, in arcmin, and multiply by 2.5 to get angular radius.
	Multiply by 2 again to get angular diameter (angular size)
	'''
	return 2.5 * r_h * 2


def get_true_diameter(angular_size, dist):
	'''Takes angular size, in arcmin, and distance from Sun, in pc,
	Calculates true diameter (in pc)
	'''
	diam_rad = angular_size * (2.9089 * 10**-4) # 2.9089 * 10^-4 rad = 1 arcmin
	return diam_rad * dist 


In [3]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 


def get_angular_size(r_h):
	'''Go  from half light radius to true radius, based on King (1966) model for half light radius.
	Takes half light radius, in arcmin, and multiply by 2.5 to get angular radius.
	Multiply by 2 again to get angular diameter (angular size)
	'''
	return 2.5 * r_h * 2


def get_true_diameter(angular_size, dist):
	'''Takes angular size, in arcmin, and distance from Sun, in pc,
	Calculates true diameter (in pc)
	'''
	diam_rad = angular_size * (2.9089 * 10**-4) # 2.9089 * 10^-4 rad = 1 arcmin
	return diam_rad * dist 


# GLOBULAR CLUSTER DATA READING
glob_data1 = pd.read_csv("glob_data.txt", skiprows = 72, nrows = 156, delim_whitespace = True, header = None)
glob_data2 = pd.read_csv("glob_data.txt", skiprows = 252, nrows = 156, delim_whitespace = True, header = None)
glob_data3 = pd.read_csv("glob_data.txt", skiprows = 433, nrows = 156, delim_whitespace = True, header = None)

# Combining the first two columns because the names were split
glob_data1["Cname"] = glob_data1.iloc[:,0] + glob_data1.iloc[:,1].astype(str)


glob_data = pd.concat([glob_data1["Cname"], glob_data1.iloc[:,4:10], glob_data1.iloc[:,10], glob_data2.iloc[:,4], glob_data3.iloc[:,8]], axis = 1)
glob_data.columns = ["Cname", "RA1", "RA2","RA3", "Dec1", "Dec2", "Dec3", "R_sun", "E(B-V)","r_h" ]


# OPEN CLUSTER DATA READING
open_data = pd.read_csv("open_data.tsv", skiprows = 39, delimiter = "\t", header = None)

# Code generated by ChatGPT-4o (OpenAI, 2024) - November 2024 version, Prompt: "I have too many whitespace missing values in my dataset, how do I remove them?"
open_data = open_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)  # Remove leading/trailing spaces
open_data.replace('', pd.NA, inplace=True)  # Replace empty strings with NaN
open_data = open_data.dropna()

open_data.columns = ["RA", "Dec", "Diam", "Dist", "Age (log years)", "Metallicity", "E(B-V)", "Cname", "_RA.icrs", "_DE.icrs"]

print("Globular Data","\n")
print(glob_data.head())

print("Open Data", "\n")
print(open_data)

Globular Data 

      Cname    RA1  RA2   RA3  Dec1    Dec2   Dec3  R_sun  E(B-V)    r_h
0    NGC104   5.67  -72   4.0  52.6  305.89 -44.89    4.5    0.04   0.36
1    NGC288  45.24  -26  34.0  57.4  152.30 -89.38    8.9    0.03   1.35
2    NGC362  14.26  -70  50.0  55.6  301.53 -46.25    8.6    0.05   0.18
3  Whiting1  57.00   -3  15.0  10.0  161.22 -60.76   30.1    0.03   7.93
4   NGC1261  16.21  -55  12.0  58.4  270.54 -52.12   16.3    0.01  17.73
Open Data 

          RA     Dec  Diam  Dist Age (log years) Metallicity E(B-V)  \
4    00 01.7  -30 12  89.0   190            7.70        0.03   0.09   
26   00 39.4  +85 04  13.0  1550            9.70       -0.06   0.04   
42   01 26.1  +63 03   4.4   900            9.10       -0.76   0.90   
60   01 54.8  +37 26  49.0   400            9.04       -0.21   0.02   
86   02 38.8  +42 34  35.0   440            8.29       -0.26   0.04   
..       ...     ...   ...   ...             ...         ...    ...   
909  20 32.5  +28 08  31.0   800     

/tmp/ipykernel_1377045/1955877692.py:23: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  glob_data1 = pd.read_csv("glob_data.txt", skiprows = 72, nrows = 156, delim_whitespace = True, header = None)
/tmp/ipykernel_1377045/1955877692.py:24: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  glob_data2 = pd.read_csv("glob_data.txt", skiprows = 252, nrows = 156, delim_whitespace = True, header = None)
/tmp/ipykernel_1377045/1955877692.py:25: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  glob_data3 = pd.read_csv("glob_data.txt", skiprows = 433, nrows = 156, delim_whitespace = True, header = None)
/tmp/ipykernel_1377045/1955877692.py:39: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map

### Calculate the true diameter (in parsecs)

In [4]:
glob_data['true diam'] = get_true_diameter(get_angular_size(glob_data['r_h']), 1000 * glob_data['R_sun'])
glob_data

,Cname,RA1,RA2,RA3,Dec1,Dec2,Dec3,R_sun,E(B-V),r_h,true diam
0,NGC104,5.67,-72,4.0,52.6,305.89,-44.89,4.5,0.04,0.36,2.356209
1,NGC288,45.24,-26,34.0,57.4,152.30,-89.38,8.9,0.03,1.35,17.475217
2,NGC362,14.26,-70,50.0,55.6,301.53,-46.25,8.6,0.05,0.18,2.251489
3,Whiting1,57.00,-3,15.0,10.0,161.22,-60.76,30.1,0.03,7.93,347.167034
4,NGC1261,16.21,-55,12.0,58.4,270.54,-52.12,16.3,0.01,17.73,420.334596
...,...,...,...,...,...,...,...,...,...,...,...
151,NGC7078,58.33,12,10.0,1.2,65.01,-27.31,10.4,0.10,0.14,2.117679
152,NGC7089,27.02,0,49.0,23.7,53.37,-35.77,11.5,0.06,0.32,5.352376
153,NGC7099,22.12,-23,10.0,47.5,27.18,-46.84,8.1,0.03,0.06,0.706863
154,Pal12,38.84,-21,15.0,9.4,30.51,-47.68,19.0,0.02,19.21,530.859706


### Logistic Regression

In [14]:
## Necessary imports
from sklearn.model_selection import train_test_split
from sklearn import metrics
import sklearn as sk

In [23]:
# setting up the dataset for logistic regression
glob_d_c = glob_data
open_d_c = open_data
glob_d_c['cluster_type'] = 1 #making globular clusters equal to one
open_d_c['cluster_type'] = 0 # making open clusters equal to two 
cluster_df = pd.concat([open_d_c,glob_d_c]) #combining the data for the model. 
cluster_df.head()

,RA,Dec,Diam,Dist,Age (log years),Metallicity,E(B-V),Cname,_RA.icrs,_DE.icrs,cluster_type,RA1,RA2,RA3,Dec1,Dec2,Dec3,R_sun,r_h,true diam
4,00 01.7,-30 12,89.0,190,7.70,0.03,0.09,C0001-302,00 04 15.3,-29 55 18,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,00 39.4,+85 04,13.0,1550,9.70,-0.06,0.04,C0039+850,00 44 22.5,+85 20 25,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,01 26.1,+63 03,4.4,900,9.10,-0.76,0.90,C0126+630,01 29 29.1,+63 18 29,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,01 54.8,+37 26,49.0,400,9.04,-0.21,0.02,C0154+374,01 57 46.7,+37 40 36,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,02 38.8,+42 34,35.0,440,8.29,-0.26,0.04,C0238+425,02 42 01.4,+42 46 46,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
## cleaning up the data frame
combined_dia = glob_d_c['true diam'].fillna(open_d_c['Diam']) #combining the diameters into one column
cluster_df['diameter'] = combined_dia # adding that column to my df
#dropping unnecessary columns into a new df.
cluster_dropped = cluster_df.drop(columns=['RA','Dec','Dist','Age (log years)','Metallicity','Cname','_RA.icrs','_DE.icrs','RA1','RA2','RA3','Dec1','Dec2','Dec3','R_sun','r_h'])
cluster_dropped.head()
# reorganizing the dataframe !
cluster_dropped = cluster_dropped[['E(B-V)','diameter','cluster_type']]
cluster_dropped.head()

,E(B-V),diameter,cluster_type
4,0.09,420.334596,0
26,0.04,9.112129,0
42,0.90,241.629233,0
60,0.02,0.299617,0
86,0.04,1.727887,0


In [48]:
# making sure they are not objects
cluster_dropped = cluster_dropped.apply(pd.to_numeric, errors='raise')

,E(B-V),diameter,cluster_type
4,0.09,420.334596,0
26,0.04,9.112129,0
42,0.90,241.629233,0
60,0.02,0.299617,0
86,0.04,1.727887,0
...,...,...,...
151,0.10,2.117679,1
152,0.06,5.352376,1
153,0.03,0.706863,1
154,0.02,530.859706,1


In [62]:
# checking for infinite or nans
print(cluster_dropped.isna().sum())

print(np.isinf(cluster_dropped).sum())

pd.set_option('display.max_rows', None)
# print(cluster_dropped['diameter'])
print(glob_data['true diam'])
print(glob_data.isna().sum())

E(B-V)           0
diameter        74
cluster_type     0
dtype: int64
E(B-V)          0
diameter        0
cluster_type    0
dtype: int64
0         2.356209
1        17.475217
2         2.251489
3       347.167034
4       420.334596
5       337.902187
6      4235.861640
7        52.505645
8       754.033422
9         1.583896
10        3.001985
11      296.882334
12       38.444022
13             NaN
14             NaN
15        3.490680
16      103.673196
17     3129.321898
18        9.264847
19     3637.851432
20             NaN
21        2.526380
22      173.608970
23      672.805299
24        8.688884
25      177.396358
26        9.112129
27       52.639454
28       17.924642
29        5.489094
30        4.764778
31      389.652973
32       33.277816
33      630.416808
34        3.054345
35       22.968674
36        2.801271
37       77.272020
38      373.248231
39        4.799685
40      189.379571
41        1.233374
42      241.629233
43      267.281368
44      102.742348
45      

In [49]:
## creating features and labels so we can train the model. 
clust = cluster_dropped # easier to type :3
features = clust.iloc[:,0:2]
label = clust['cluster_type']


In [50]:
## defining the model
train_vectors,test_vectors, train_labels, test_labels = sk.model_selection.train_test_split(features,label,test_size=.25,train_size=.75,random_state=0)
# making sure they are the same size
print(len(train_vectors))
print(len(test_vectors))
print(len(train_labels))
print(len(test_labels))

176
59
176
59


In [51]:
## Training the model
logit_model = sm.Logit(train_labels, sm.add_constant(train_vectors))
result = logit_model.fit()
print(result.summary() )

MissingDataError: exog contains inf or nans